In [123]:
import numpy as np
import pandas as pd

In [2]:
ratings_set_df = pd.read_csv("./data_movie_lens_100k/ratings_all_development_set.csv")
users_df = pd.read_csv("./data_movie_lens_100k/user_info.csv")
movies_df = pd.read_csv("./data_movie_lens_100k/movie_info.csv")

In [159]:
print(ratings_set_df)

       user_id  item_id  rating
0          772       36       3
1          471      228       5
2          641      401       4
3          312       98       4
4           58      504       5
...        ...      ...     ...
89987      415      813       4
89988      842      120       3
89989      574      505       2
89990      757      472       5
89991      503      204       3

[89992 rows x 3 columns]


In [158]:
print(users_df)

     user_id  age  is_male  orig_user_id
0          0   24        1             1
1          1   53        0             2
2          2   23        1             3
3          3   24        1             4
4          4   33        0             5
..       ...  ...      ...           ...
938      938   26        0           939
939      939   32        1           940
940      940   20        1           941
941      941   48        0           942
942      942   22        1           943

[943 rows x 4 columns]


In [160]:
print(movies_df)

      item_id                                      title  release_year  \
0           0                           Toy Story (1995)          1995   
1           1                           GoldenEye (1995)          1995   
2           2                          Four Rooms (1995)          1995   
3           3                          Get Shorty (1995)          1995   
4           4                             Copycat (1995)          1995   
...       ...                                        ...           ...   
1676     1676                          Mat' i syn (1997)          1998   
1677     1677                           B. Monkey (1998)          1998   
1678     1678                       Sliding Doors (1998)          1998   
1679     1679                        You So Crazy (1994)          1994   
1680     1680  Scream of Stone (Schrei aus Stein) (1991)          1996   

      orig_item_id  
0                1  
1                2  
2                3  
3                4  
4     

In [157]:
from surprise import Dataset, NormalPredictor, Reader, SVD, accuracy


from train_valid_test_loader import load_train_valid_test_datasets

# Load the dataset in the same way as the main problem 
train_tuple, valid_tuple, test_tuple, n_users, n_items = \
        load_train_valid_test_datasets()


def tuple_to_surprise_dataset(tupl):
    """
    This function convert a subset in the tuple form to a `surprise` dataset. 
    """
    ratings_dict = {
        "userID": tupl[0],
        "itemID": tupl[1],
        "rating": tupl[2],
    }

    df = pd.DataFrame(ratings_dict)

    # A reader is still needed but only the rating_scale param is requiered.
    reader = Reader(rating_scale=(1, 5))

    # The columns must correspond to user id, item id and ratings (in that order).
    dataset = Dataset.load_from_df(df[["userID", "itemID", "rating"]], reader)

    return dataset


trainset = tuple_to_surprise_dataset(train_tuple).build_full_trainset()

algo = SVD(n_factors=10)

algo.fit(trainset)

# uid = valid_tuple[0][0]
# iid = valid_tuple[1][0]
# rui = valid_tuple[2][0]

# mu = algo.trainset.global_mean # SVD does not even fit mu -- it directly use the rating mean 
# bu = algo.bu[trainset.to_inner_uid(uid)]
# bi = algo.bi[trainset.to_inner_iid(iid)] 
# pu = algo.pu[trainset.to_inner_uid(uid)] 
# qi = algo.qi[trainset.to_inner_iid(iid)]

user_vectors = algo.pu
item_vectors = algo.qi

print(user_vectors.shape)
print(item_vectors.shape)

user_features = users_df[['age', 'is_male']].values
item_features = movies_df[['release_year']].values

print(user_features.shape)
print(item_features.shape)


# user_vectors = np.resize(user_vectors, (item_features.shape[0], user_vectors.shape[1]))
# item_vectors = np.resize(item_vectors, (item_features.shape[0], item_vectors.shape[1]))

# user_features = np.resize(user_features, (item_features.shape[0], user_features.shape[1]))


# print(user_features)

# print(item_features.shape)

# feature_vectors = np.concatenate((user_vectors, item_vectors, user_features, item_features), axis=1)
#print(feature_vectors)




(943, 10)
(1626, 10)
(943, 2)
(1681, 1)


In [127]:
ratings = ratings_set_df['rating'].values

labels = (ratings >= 4.5).astype(int)

SEED = 12345

X_train, X_test, y_train, y_test = train_test_split(feature_vectors, labels, test_size=0.25, random_state=SEED)

(89992,)


ValueError: Found input variables with inconsistent numbers of samples: [1681, 89992]